<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gradient Without Backpropagation

In [10]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

In [15]:
#Load the dataset

penguins = tfds.load('penguins', as_supervised=True, split='train')

X, y = penguins.batch(1000).get_single_element()
X, y = X.numpy(), y.numpy()

In [16]:
#Split the data into train and test data

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

In [17]:
X.shape

(334, 4)

In [21]:
#Initialize parameters

def init():
  theta = torch.randn(X.shape, requires_grad=False)
  v = torch.randn(X.shape, requires_grad=False)
  return theta, v